In [1]:
# Importing Libraries
import pandas as pd
import numpy as np

In [19]:
# Importing Dataset
Train = pd.read_csv('07_Train_UWu5bXk.csv')
Test = pd.read_csv('07_Test_u94Q5KV.csv')
Train.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [5]:
# Splitting into Training & Testing
X_train, y_train = Train.iloc[:, :-1].values, Train.iloc[:, -1].values

In [6]:
# Identifying Categorical Variables, NaN Values
print(Train.info())
print(Train.isnull().any())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
Item_Identifier              8523 non-null object
Item_Weight                  7060 non-null float64
Item_Fat_Content             8523 non-null object
Item_Visibility              8523 non-null float64
Item_Type                    8523 non-null object
Item_MRP                     8523 non-null float64
Outlet_Identifier            8523 non-null object
Outlet_Establishment_Year    8523 non-null int64
Outlet_Size                  6113 non-null object
Outlet_Location_Type         8523 non-null object
Outlet_Type                  8523 non-null object
Item_Outlet_Sales            8523 non-null float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.1+ KB
None
Item_Identifier              False
Item_Weight                   True
Item_Fat_Content             False
Item_Visibility              False
Item_Type                    False
Item_MRP                     False
Outlet_Iden

In [25]:
# Categorical Variables Indexes:    0, 2, 4, 6, 8, 9, 10
# Missing Values in Indexes:        1, 8
# Deformed Values in Indexes:       2
def prepro(dataset):
    X_dataset = dataset.values
    # A. Imputing Missing Values
    from sklearn.preprocessing import Imputer
    impute = Imputer(missing_values='NaN', strategy='mean')
    column1 = X_dataset[:, 1].reshape(-1, 1)
    X_dataset[:, 1] = impute.fit_transform(column1).reshape(1, -1)
    # B. Deformed Values into Uniform Values
    X_dataset = pd.DataFrame(X_dataset)
    X_dataset.iloc[:, 2] = X_dataset.iloc[:, 2].map({'reg': 'Regular', 'LF': 'Low Fat', 'low fat': 'Low Fat','Low Fat': 'Low Fat', 'Regular': 'Regular'})
    X_dataset = X_dataset.values
    # Label Encoding for column2
    from sklearn.preprocessing import LabelEncoder, OneHotEncoder
    label_encoder_1 = LabelEncoder()
    X_dataset[:, 2] = label_encoder_1.fit_transform(X_dataset[:, 2])
    label_encoder_2 = LabelEncoder()
    X_dataset[:, 10] = label_encoder_2.fit_transform(X_dataset[:, 10])
    label_encoder_3 = LabelEncoder()
    X_dataset[:, 9] = label_encoder_2.fit_transform(X_dataset[:, 9])
    # C. Removal of unused Variables
    X_dataset = X_dataset[:, [1, 2, 3, 5, 7, 9, 10]]

    # D. Hot Encoding
    ohe_1 = OneHotEncoder(categorical_features=[5, 6])
    X_dataset = ohe_1.fit_transform(X_dataset).toarray()
    return X_dataset

In [ ]:
# Machine: Regressor | Regressor: Linear
X_train = prepro(Train)

from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)
X_test = prepro(dataset=Test)
y_pred = lin_reg.predict(X_test)

X_df = pd.DataFrame(X_test)
y_df = pd.DataFrame(y_pred)
Headers = pd.DataFrame(Test.iloc[:, [0, 6]])

Result = pd.concat([Headers, X_df, y_df], axis=1, ignore_index=True)
Result = Result.rename(columns={0: 'Item Identifier', 1: 'Outlet Identifier', 14: 'Predicted Outlet Sales'})
Result.to_csv('07_BigMartSales_RESULTS.csv', index=False)
print('CHECK FOR: 07_BigMartSales_RESULTS.csv IN WORKING DIRECTORY')